# 

# Automating processes:
----
(interactive version -> *.ipynb*)

## Importing libraries:

In [ ]:
import numpy as np
import pandas as pd
import selenium as sn

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from dotenv import load_dotenv
import os

load_dotenv()

link = os.getenv('LINK')
user_login = os.getenv('USER_LOGIN')
user_pw = os.getenv('USER_PASSWORD')

Setting up driver for Google Chrome:

In [ ]:
# Checks if the webdriver is installed and up to date
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# Opens the link stored in .env
driver.get(link)

# Fill the login field
login = driver.find_element(By.ID, 'txtLogin')
login.send_keys(user_login)

# Fill the password field
login = driver.find_element(By.ID, 'txtSenha')
login.send_keys(user_pw)

# Click the enter button
enter_btn = driver.find_element(By.ID, 'btnOk')
enter_btn.click()

# Enter the 'Recuperação Ambiental' module and 'Controle' submodule
module_btn = driver.find_element(By.ID, 'imbModuloP3')
driver.execute_script("arguments[0].click();", module_btn)
sub_module_btn = driver.find_element(By.ID, 'imbModuloP3')
sub_module_btn = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.CSS_SELECTOR, "div.rollfx[onclick*='DefaultRecuperacaoAmbientalControle.aspx']"))
)
driver.execute_script("arguments[0].click();", sub_module_btn)

# Wait until the new page is completely loaded
WebDriverWait(driver, 10).until(
    lambda driver: driver.execute_script('return document.readyState') == 'complete'
)

# Enter the module 'Controle > Boletim de Silvicultura'
# Click on first element ('Controle') and wait until the 
# submodule ('Boletim de Silvicultura') is visible, then click it
first_element = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.CSS_SELECTOR, "td.MenuItem[id='Menu1-menuItem000']"))
)
first_element.click()

# Wait until the second element is visible and clickable
second_element = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.CSS_SELECTOR, "td.MenuItem[id='Menu1-menuItem000-subMenu-menuItem000']"))
)
second_element.click()

# Wait until the new page is completely loaded
WebDriverWait(driver, 10).until(
    lambda driver: driver.execute_script('return document.readyState') == 'complete'
)

# Select the planned services
radio_button = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.ID, 'rblApontamento_0'))
)
driver.execute_script("arguments[0].click();", radio_button)
